In [ ]:
import requests
import pandas as pd

url = "https://data.ny.gov/resource/wujg-7c2s.json"

all_data = []
offset = 0
batch_size = 50_000

while True:
    query = f"""
        SELECT
            DISTINCT(station_complex_id),
            station_complex,
            borough,
            latitude,
            longitude
        WHERE transit_timestamp = '2024-01-01T00:00:00'
        LIMIT {batch_size} OFFSET {offset}
    """

    params = {"$query": query}

    r = requests.get(url, params=params, timeout=60)
    r.raise_for_status()
    data = r.json()

    if not data:
        break

    all_data.extend(data)
    print(f"Leidos {len(data):,} registros (offset {offset:,})")

    
    if len(data) < batch_size:
        break

    offset += batch_size

df = pd.DataFrame(all_data)
print(f"\nTotal de filas: {len(df):,}")
print(df.head())

In [ ]:
display(df)

In [ ]:
df.to_csv("dim_mta_metro_ridership.csv", index=False)
print("Archivo guardado")